<a href="https://colab.research.google.com/github/Debottam/tensorFlowCustomized/blob/master/customizedTensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

print("tensorflow: ", tf.__version__)
print("keras: ", keras.__version__)

tensorflow:  2.3.0
keras:  2.4.0


In [2]:
#customized error
def mse_fn(y_true, y_pred):
  error = y_true - y_pred
  mserror = tf.reduce_mean(tf.square(error))
  return mserror

In [3]:
#data load and preprocessing
(X_train, Y_train), (X_test, Y_test) = keras.datasets.mnist.load_data()
Y_train = np.array(Y_train).astype(np.float32)
Y_test = np.array(Y_test).astype(np.float32)
X_train = X_train/255.
X_test = X_test/255.

11493376/11490434 [==============================] - 0s 0us/step


In [6]:
# customized DNN layer
class MyDense(keras.layers.Layer):
  def __init__(self, units, activation=None, **kwargs):
    super().__init__(**kwargs)
    self.units = units
    self.activation = keras.activations.get(activation)
  
  def build(self, batch_input_shape):
    self.kernel = self.add_weight(name = "kernel", shape=[batch_input_shape[-1], self.units], initializer="glorot_normal")
    self.bias = self.add_weight(name = "bias", shape=[self.units], initializer = 'zeros')
    super().build(batch_input_shape)

  def call(self, X):
    return self.activation(X @ self.kernel + self.bias)
  
  def compute_output_shape(self, batch_input_shape):
    return tf.TensorShape(batch_input_shape.as_list()[:-1]+[self.units])
  
  def get_config(self):
    base_config = super().get_config()
    return {**base_config, "units":self.units, "activation": keras.activations.serialize(self.activation)}

In [7]:
# the Model
class DNN_model(keras.Model):
  def __init__(self, output_dim, **kwargs):
    super().__init__(**kwargs)
    self.hidden1 = keras.layers.Flatten(input_shape = [28, 28])
    self.hidden2 = MyDense(32, activation='relu')
    self.hidden3 = MyDense(16, activation='relu')
    self.hidden4 = MyDense(8, activation='relu')
    self.hidden5 = MyDense(output_dim, activation='relu')
  
  def call(self, inputs):
    Z = self.hidden1(inputs)
    Z = self.hidden2(Z)
    Z = self.hidden3(Z)
    Z = self.hidden4(Z)
    Z = self.hidden5(Z)
    return Z


In [8]:
# compilation without custom loop
model_cust_loss = DNN_model(1)
model_cust_loss.compile(optimizer='adam', loss = mse_fn, metrics=['accuracy'])
model_cust_loss.fit(X_train, Y_train, epochs=20 , batch_size=32)

Epoch 1/20
1875/1875 [==============================] - 2s 1ms/step - loss: 2.2424 - accuracy: 0.1735
Epoch 2/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.9630 - accuracy: 0.2007
Epoch 3/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.7705 - accuracy: 0.2032
Epoch 4/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.6615 - accuracy: 0.2047
Epoch 5/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.5978 - accuracy: 0.2059
Epoch 6/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.5439 - accuracy: 0.2064
Epoch 7/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4953 - accuracy: 0.2071
Epoch 8/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4583 - accuracy: 0.2072
Epoch 9/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4268 - accuracy: 0.2076
Epoch 10/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.403

In [9]:
# Customized tensorflow loop
model_cust_loss = DNN_model(1)

def random_batch(X, y, batch_size=32):
  idx = np.random.randint(len(X), size=batch_size)
  return X[idx], y[idx]

n_epochs = 5
batch_size = 32
n_steps = len(X_train)//batch_size
optimizer = keras.optimizers.Adam() 
print("n_steps: ", n_steps)

epoch_losses = []
for epoch in range (1,n_epochs+1):
  print("Epoch {}/{}".format(epoch, n_epochs))
  batch_losses = []
  for step in range(1, n_steps+1):
    X_batch, Y_batch = random_batch(X_train, Y_train, batch_size = 32)
    with tf.GradientTape() as tape:
      Y_pred = model_cust_loss(X_batch, training=True)
      current_loss = mse_fn(Y_batch, Y_pred) 
    batch_losses.append(current_loss)
    gradients = tape.gradient(current_loss, model_cust_loss.trainable_variables)
    optimizer.apply_gradients(zip(gradients,model_cust_loss.trainable_variables))
  epoch_losses.append(np.mean(batch_losses))
  print("loss per epoch:", epoch_losses[epoch-1])

n_steps:  1875
Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

loss per epoch: 8.842264
Epoch 2/5
loss per epoch: 8.431509
Epoch 3/5
loss per epoch: 8.326578
Epoch 4/5
loss per epoch: 8.392796
Epoch 5/5
loss per epoch: 8.358618
